In [ ]:
import gzip
import json
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
import torch.nn.functional as F
import torch.optim as optim

In [ ]:
from google.colab import drive
drive.mount("/content/drive/")

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
!pip install torch_xla[tpu] -f https://storage.googleapis.com/libtpu-releases/index.html

Looking in links: https://storage.googleapis.com/libtpu-releases/index.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.2/123.2 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.6/90.6 MB 8.9 MB/s eta 0:00:00


In [ ]:
from transformers import AutoModel, AutoTokenizer
from tokenization_bitnet import BitnetTokenizer
tokenizer = BitnetTokenizer.from_pretrained("1bitLLM/bitnet_b1_58-xl")
from transformers import AutoConfig
config = AutoConfig.from_pretrained("1bitLLM/bitnet_b1_58-xl")


model_name = "1bitLLM/bitnet_b1_58-xl"
# tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

config.save_pretrained("./configi")
tokenizer.save_pretrained("./bitnet_b1_58-xl")
model.save_pretrained("./bitnet_b1_58-xl")

/usr/local/lib/python3.10/dist-packages/torch_xla/__init__.py:253: UserWarning: `tensorflow` can conflict with `torch-xla`. Prefer `tensorflow-cpu` when using PyTorch/XLA. To silence this warning, `pip uninstall -y tensorflow && pip install tensorflow-cpu`. If you are in a notebook environment such as Colab or Kaggle, restart your notebook runtime afterwards.
  warnings.warn(


ModuleNotFoundError: No module named 'tokenization_bitnet'

In [ ]:

import json

with open("/content/drive/MyDrive/Certificates/nvdcve-1.1-2024.json", "rt", encoding="utf-8") as f:
    data = json.load(f)

# print(data["CVE_Items"])  # List of CVEs
# print(data["CVE_data_type"])  # Type of data
# print(data["CVE_data_format"])  # Format of data
# print(data["CVE_data_version"])  # Version of data
number = data["CVE_data_numberOfCVEs"] # Number of CVEs of CVEs

score = []
description = []
targets = []
import matplotlib.pyplot as plt


for i in range(int(number)):
    if(data["CVE_Items"][i]["impact"] != {}):
        score.append(data["CVE_Items"][i]["impact"]["baseMetricV3"]["cvssV3"]["baseScore"])
        targets.append(data["CVE_Items"][i]["impact"]["baseMetricV3"]["exploitabilityScore"])
        description.append(data["CVE_Items"][i]["cve"]["description"]["description_data"][0]["value"])

print(len(score))
print(len(description))

13107
13107


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
for i,s,d in zip(score, targets, description):
  print(i,s,d)
  break

9.8 3.9 A condition exists in FlashArray Purity whereby a local account intended for initial array configuration remains active potentially allowing a malicious actor to gain elevated privileges.


In [ ]:
!pip uninstall tensorflow
!pip install tensorflow-cpu


Found existing installation: tensorflow 2.17.1
Uninstalling tensorflow-2.17.1:
  Would remove:
    /usr/local/bin/import_pb_to_tensorboard
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.10/dist-packages/tensorflow-2.17.1.dist-info/*
    /usr/local/lib/python3.10/dist-packages/tensorflow/*
Proceed (Y/n)? y
ERROR: Operation cancelled by user
Traceback (most recent call last):
  File "/usr/lib/python3.10/shutil.py", line 816, in move
    os.rename(src, real_dst)
OSError: [Errno 18] Invalid cross-device link: '/usr/local/lib/python3.10/dist-packages/tensorflow/' -> '/usr/local/lib/python3.10/dist-packages/~ensorflow'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/base_command.py", line 179, in exc_

In [ ]:
!pip cache purge
!pip install --upgrade transformers


In [ ]:
# Load model directly
from transformers import AutoTokenizer , AutoModelForMaskedLM
tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")
# model = AutoModelForCausalLM.from_pretrained()

# Load the model and resize embeddings to match new tokenizer size
model = AutoModelForMaskedLM.from_pretrained("distilbert/distilbert-base-uncased", num_labels=1)
model.resize_token_embeddings(len(tokenizer))

Embedding(30522, 768, padding_idx=0)

In [ ]:
# # Define a custom Dataset class
# class CVEDataset(Dataset):
#     def __init__(self, descriptions, scores, exploit_scores, tokenizer, max_length=128):
#         self.descriptions = descriptions
#         self.scores = scores
#         self.exploit_scores = exploit_scores
#         self.tokenizer = tokenizer
#         self.max_length = max_length

#     def __len__(self):
#         return len(self.descriptions)

#     def __getitem__(self, idx):
#         # Tokenize the description
#         tokens = self.tokenizer(
#             self.descriptions[idx],
#             truncation=True,
#             padding="max_length",
#             max_length=self.max_length,
#             return_tensors="pt",
#         )

#         # CVSS Score as an additional feature
#         cvss_score = torch.tensor(self.scores[idx], dtype=torch.float32)

#         # Exploitability Score as the target
#         target = torch.tensor(self.exploit_scores[idx], dtype=torch.float32)

#         # Combine tokenized inputs and CVSS score
#         inputs = {
#             "input_ids": tokens["input_ids"].squeeze(0),
#             "attention_mask": tokens["attention_mask"].squeeze(0),
#             "cvss_score": cvss_score,
#             "labels": target,
#         }

#         return inputs

In [ ]:
# Dataset class
class VulnerabilityDataset(Dataset):
    def __init__(self, descriptions, scores, targets):
        self.descriptions = descriptions
        self.scores = scores
        self.targets = targets

    def __len__(self):
        return len(self.descriptions)

    def __getitem__(self, idx):
        text = self.descriptions[idx]
        score = self.scores[idx]
        target = self.targets[idx]
        return {"text": text, "cvss_score": score, "labels": target}

In [ ]:
# Prepare data
train_descriptions = description  # Your CVE descriptions
train_scores = score  # CVSS scores
train_targets = targets  # Exploitability scores

train_dataset = VulnerabilityDataset(train_descriptions, train_scores, train_targets)
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)

# Optimizer and loss
optimizer = optim.AdamW(model.parameters(), lr=5e-5)
loss_fn = torch.nn.MSELoss()

# Training loop
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

DistilBertForMaskedLM(
  (activation): GELUActivation()
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0

In [ ]:
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    model.resize_token_embeddings(len(tokenizer))


In [ ]:
print("Tokenizer vocab size:", tokenizer.vocab_size)
sample_text = "Test CVE description"
encoding = tokenizer(sample_text, return_tensors="pt")
print("Token IDs:", encoding['input_ids'])
print("Max token ID:", torch.max(encoding['input_ids']))


In [ ]:
import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import DataLoader
from tqdm import tqdm  # Progress bar
import torch_xla
import torch_xla.core.xla_model as xm  # XLA support
import torch_xla.distributed.parallel_loader as pl  # Parallel data loader for TPU
import torch_xla.utils.utils as xu
import torch_xla.debug.metrics as met
import torch_xla.distributed.xla_multiprocessing as xmp

ModuleNotFoundError: No module named 'torch_xla'

In [ ]:


# Function to train on TPU
def train_model(rank, flags):
    # Prepare TPU device
    device = xm.xla_device()

    # Prepare data
    train_descriptions = description  # Your CVE descriptions
    train_scores = score  # CVSS scores
    train_targets = exploit  # Exploitability scores

    # Dataset and DataLoader
    train_dataset = VulnerabilityDataset(train_descriptions, train_scores, train_targets)
    train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)

    # Optimizer and loss
    model.to(device)
    optimizer = optim.AdamW(model.parameters(), lr=5e-5)
    loss_fn = nn.MSELoss()

    epochs = 3
    for epoch in range(epochs):
        model.train()
        train_loss = 0.0

        # Wrap DataLoader with ParallelLoader for TPU support
        para_loader = pl.MpDeviceLoader(train_loader, device)
        for batch in tqdm(para_loader, desc=f"Epoch {epoch+1}/{epochs} [Training]"):
            optimizer.zero_grad()

            texts = batch['text']
            cvss_scores = batch['cvss_score'].clone().detach().unsqueeze(1).to(device)
            labels = batch['labels'].clone().detach().unsqueeze(1).to(device)

            # Tokenize input text
            encoding = tokenizer(
                texts,
                padding=True,
                truncation=True,
                max_length=128,
                return_tensors="pt"
            ).to(device)

            # Forward pass
            outputs = model(**encoding)
            output_layer = torch.nn.Linear(outputs, out_features=1)  # Outputs (batch_size, 1)

            logits = output_layer.logits.squeeze(-1) + cvss_scores  # Combine CVSS score with model output

            loss = loss_fn(logits, labels)
            loss.backward()
            xm.optimizer_step(optimizer)  # XLA-specific step for optimization

            train_loss += loss.item()

        avg_train_loss = train_loss / len(train_loader)
        print(f"Epoch {epoch+1}/{epochs}, Train Loss: {avg_train_loss:.4f}")

        # Log TPU metrics for debugging
        xm.master_print(met.metrics_report())

    # Save the model
    xm.save(model.state_dict(), "bitnet_exploit_model.pth")
    tokenizer.save_pretrained("bitnet_exploit_model")


# Start training on TPU using xmp.spawn
if __name__ == '__main__':
    flags = {}
    xmp.spawn(train_model, args=(flags,), nprocs=8, start_method='fork')


NameError: name 'xmp' is not defined

In [ ]:
epochs = 3
for epoch in range(epochs):
    model.train()
    train_loss = 0.0

    for batch in train_loader:
        optimizer.zero_grad()

        texts = batch['text']
        cvss_scores = batch['cvss_score'].clone().detach().unsqueeze(1).to(device)
        labels = batch['labels'].clone().detach().unsqueeze(1).to(device)


        # Tokenize input text
        encoding = tokenizer(
            texts,
            padding=True,
            truncation=True,
            max_length=128,
            return_tensors="pt"
        ).to(device)

        print(len(encoding['input_ids']))

        outputs = model(**encoding)
        logits = outputs.logits.squeeze(-1) + cvss_scores  # Combine CVSS score with model output

        loss = loss_fn(logits, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    avg_train_loss = train_loss / len(train_loader)
    print(f"Epoch {epoch+1}/{epochs}, Train Loss: {avg_train_loss:.4f}")

# Save the model
model.save_pretrained("bitnet_exploit_model")
tokenizer.save_pretrained("bitnet_exploit_model")

In [ ]:
from tqdm import tqdm  # Import tqdm
import time  # For time estimation if needed

epochs = 3
for epoch in range(epochs):
    model.train()
    train_loss = 0.0

    # Add tqdm progress bar for training
    train_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs} [Training]", leave=False)
    for batch in train_bar:
        optimizer.zero_grad()

        # Data processing
        texts = batch['text']
        cvss_scores = batch['cvss_score'].clone().detach().unsqueeze(1).to(device)
        labels = batch['labels'].clone().detach().unsqueeze(1).to(device)
        encoding = tokenizer(texts, padding=True, truncation=True, max_length=128, return_tensors="pt").to(device)

        # print(encoding)

        # Forward pass
        outputs = model(**encoding)
        logits = outputs.logits.squeeze(-1) + cvss_scores
        loss = loss_fn(logits, labels)

        # Backward pass
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

        # Update tqdm with current loss
        train_bar.set_postfix({"Loss": f"{loss.item():.4f}"})

    avg_train_loss = train_loss / len(train_loader)

    # Validation Loop with Progress Bar
    model.eval()
    val_loss = 0.0
    val_bar = tqdm(val_loader, desc=f"Epoch {epoch+1}/{epochs} [Validation]", leave=False)
    with torch.no_grad():
        for batch in val_bar:
            texts = batch['text']
            cvss_scores = batch['cvss_score'].clone().detach().unsqueeze(1).to(device)
            labels = batch['labels'].clone().detach().unsqueeze(1).to(device)
            encoding = tokenizer(texts, padding=True, truncation=True, max_length=128, return_tensors="pt").to(device)

            # print(encoding.shape())

            # Forward pass
            outputs = model(**encoding)
            logits = outputs.logits.squeeze(-1) + cvss_scores
            loss = loss_fn(logits, labels)
            val_loss += loss.item()

            # Update tqdm with current loss
            val_bar.set_postfix({"Loss": f"{loss.item():.4f}"})

    avg_val_loss = val_loss / len(val_loader)

    print(f"Epoch {epoch+1}/{epochs}")
    print(f"Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}")

# Save the model
model.save_pretrained("bitnet_exploit_model")
tokenizer.save_pretrained("bitnet_exploit_model")


In [ ]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np
scaler = MinMaxScaler()
# score = scaler.fit_transform(score)  # Scale numerical features
# score = scaler.fit_transform(score)  # Scale numerical features
targets = scaler.fit_transform(np.array(targets).reshape(-1, 1)).squeeze()  # Scale targets
score = scaler.fit_transform(np.array(score).reshape(-1, 1)).squeeze()  # Scale targets

In [ ]:
import random
hell = zip(score, targets, description)
print(len(hell))
for i,t,s in random.sample(hell, 20):
  print(i,t,s)
  break

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from transformers import AutoTokenizer, AutoModelForCausalLM
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Dataset

# Dummy dataset class
class NVDDataset(Dataset):
    def __init__(self, descriptions, cvss_scores, targets, tokenizer, max_length):
        self.descriptions = descriptions
        self.cvss_scores = cvss_scores
        self.targets = targets
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.descriptions)

    def __getitem__(self, idx):
        # Tokenize text description
        encoding = self.tokenizer(
            self.descriptions[idx],
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt'
        )
        return {
            'input_ids': encoding['input_ids'].squeeze(0),
            'attention_mask': encoding['attention_mask'].squeeze(0),
            'cvss_scores': torch.tensor(self.cvss_scores[idx], dtype=torch.float32),
            'target': torch.tensor(self.targets[idx], dtype=torch.float32)
        }

# Simple Model Class
class NVDRegressionModel(nn.Module):
    def __init__(self, pretrained_model_name, config,num_numerical_features):
        super(NVDRegressionModel, self).__init__()
        # model = LlamaForCausalLM.from_pretrained("meta-llama/Bitnet-2-7b-hf")
        # self.transformer = LlamaForCausalLM.from_pretrained(pretrained_model_name, config=config)
        self.transformer = AutoModelForCausalLM.from_pretrained(pretrained_model_name, config=config)
        self.linear = nn.Linear(self.transformer.config.hidden_size + num_numerical_features, 1)

    def forward(self, input_ids, attention_mask, cvss_scores):
        # print("Input IDs shape:", input_ids.shape)
        # print("Attention mask shape:", attention_mask.shape)
        # print("Numerical features shape:", cvss_scores.shape)
        transformer_outputs = self.transformer(input_ids=input_ids, attention_mask=attention_mask, output_attentions=True)
        cls_embedding = transformer_outputs.last_hidden_state
        # CLS token representation
        # print("CLS Embedding shape:", cls_embedding.shape)
        # print("CLS Embedding min/max:", cls_embedding.min().item(), cls_embedding.max().item())
        # print("CLS Embedding (first 5 rows):", cls_embedding[:5])
        # transformer_outputs = model.transformer(input_ids=input_ids, attention_mask=attention_mask, output_attentions=True)
        attention_scores = transformer_outputs.attentions  # List of attention tensors
        # for i, scores in enumerate(attention_scores):
          # print(f"Layer {i} Attention min/max: {scores.min()}, {scores.max()}")

        combined_features = torch.cat((cls_embedding, cvss_scores.unsqueeze(1)), dim=1)
        # combined_features = torch.cat((cls_embedding, cvss_scores), dim=1)  # Combine embeddings and numerical features
        output = self.linear(combined_features)
        return output.squeeze()

from torch.cuda.amp import autocast, GradScaler
scaler = GradScaler()
from torch.utils.checkpoint import checkpoint

# Enable gradient checkpointing



# Training Function
def train(model, dataloader, optimizer, loss_fn, device):
    model.transformer.gradient_checkpointing_enable()
    model.train()
    total_loss = 0
    for batch in tqdm(dataloader, desc="Processing", total=len(dataloader)):
        input_ids = batch['input_ids'].to(device)
        # print("Max Token ID:", input_ids.max().item())
        # print("Min Token ID:", input_ids.min().item())

        attention_mask = batch['attention_mask'].to(device)
        cvss_scores = batch['cvss_scores'].to(device)
        targets = batch['target'].to(device)
        # print(input_ids.shape)
        # print(attention_mask.shape)
        # print(cvss_scores.shape)
        # print(targets.shape)


        print("CVSS Scores:", cvss_scores[:5])
        print("Targets:", targets[:5])

        cvss_scores = torch.nan_to_num(cvss_scores, nan=0.0, posinf=1.0, neginf=0.0)
        targets = torch.nan_to_num(targets, nan=0.0, posinf=1.0, neginf=0.0)

        optimizer.zero_grad()


        # predictions = model(input_ids, attention_mask, cvss_scores)
        # loss = loss_fn(predictions, targets)
    #     loss.backward()
    #     optimizer.step()

    #     total_loss += loss.item()
    # return total_loss / len(dataloader)

    # Use autocast for mixed precision
        with autocast():
            predictions = model(input_ids, attention_mask, cvss_scores)
            loss = loss_fn(predictions, targets)
            print(loss)

        # Scale the loss and perform backward pass
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        total_loss += loss.item()
    return total_loss / len(dataloader)

# Evaluation Function
def evaluate(model, dataloader, device):
    model.eval()
    predictions, actuals = [], []
    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            cvss_scores = batch['cvss_scores'].to(device)
            targets = batch['target'].to(device)

            preds = model(input_ids, attention_mask, cvss_scores)
            predictions.extend(preds.cpu().numpy())
            actuals.extend(targets.cpu().numpy())
    return predictions, actuals

<ipython-input-18-114a606cd0ee>:66: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()


In [ ]:
import torch
torch.cuda.empty_cache()
torch.cuda.reset_peak_memory_stats()


In [ ]:
!export CUDA_LAUNCH_BLOCKING=1


In [ ]:
# Freeze most layers of the model
def freeze_model_layers(model, num_layers_to_freeze=None):
    """
    Freeze the layers of the model.
    If num_layers_to_freeze is None, all layers are frozen except the last few.
    """
    # If num_layers_to_freeze is not provided, freeze all layers except the final ones
    if num_layers_to_freeze is None:
        num_layers_to_freeze = len(model.transformer.h) - 2  # Keep the last two layers trainable

    # Freeze the embeddings
    for param in model.transformer.wte.parameters():
        param.requires_grad = False
    for param in model.transformer.wpe.parameters():
        param.requires_grad = False

    # Freeze the first `num_layers_to_freeze` transformer blocks
    for layer in model.transformer.h[:num_layers_to_freeze]:
        for param in layer.parameters():
            param.requires_grad = False

    # Print the layers that are frozen
    print(f"Froze {num_layers_to_freeze} layers of the transformer.")




In [ ]:
# Freeze most layers of the model
def freeze_model_layers(model, num_layers_to_freeze=None):
    """
    Freeze the layers of the model.
    If num_layers_to_freeze is None, freeze all layers except the last few.
    """
    # Inspect the model's architecture
    for name, module in model.named_children():
        print(f"Layer: {name}, Type: {type(module)}")

    # Find transformer blocks dynamically
    transformer_blocks = None
    for name, module in model.named_children():
        if "transformer" in name.lower():
            transformer_blocks = module
            break

    if transformer_blocks is None:
        raise ValueError("Cannot find transformer blocks in the model.")

    # Dynamically access the layers within the transformer
    transformer_layers = list(transformer_blocks.children())
    total_layers = len(transformer_layers)
    print(total_layers)
    if num_layers_to_freeze is None:
        num_layers_to_freeze = total_layers - 2  # Default: Freeze all but the last 2 layers

    # Freeze specified number of layers
    for i, layer in enumerate(transformer_layers[:num_layers_to_freeze]):
        for param in layer.parameters():
            param.requires_grad = False

    print(f"Froze {num_layers_to_freeze} out of {total_layers} transformer layers.")


In [ ]:
from transformers import AutoTokenizer , AutoModelForSequenceClassification
# from tokenization_bitnet import BitnetTokenizer
from tqdm import tqdm
# Main pipeline
if __name__ == "__main__":
    # Hyperparameters
    # PRETRAINED_MODEL_NAME = "distilbert-base-uncased"
    PRETRAINED_MODEL_NAME = "TinyLlama/TinyLlama-1.1B-intermediate-step-1431k-3T"
    MAX_LENGTH = 64
    BATCH_SIZE = 32
    EPOCHS = 3
    LR = 1e-5
    DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
    # DEVICE = "cpu"

    # Sample data (replace with your actual dataset)
    # descriptions = ["Sample CVE description 1", "Sample CVE description 2"] * 100
    # cvss_scores = [[5.4], [7.8]] * 100  # Numerical features
    # targets = [0.5, 0.8] * 100  # Exploitability score

    from transformers import AutoConfig
    config = AutoConfig.from_pretrained(PRETRAINED_MODEL_NAME)
    config.use_cache = False  # Disable caching

    descriptions = description
    cvss_scores = score
    targets = targets

    # Tokenizer and Dataset
    tokenizer = AutoTokenizer.from_pretrained(PRETRAINED_MODEL_NAME)
    # Set an existing token as the pad token
    if tokenizer.pad_token is None:
      tokenizer.pad_token = tokenizer.eos_token  # Use the end-of-sequence token as padding

    train_desc, val_desc, train_cvss, val_cvss, train_targets, val_targets = train_test_split(
        descriptions, cvss_scores, targets, test_size=0.2, random_state=42
    )

    print("Tokenizer vocab size:", tokenizer.vocab_size)


    train_dataset = NVDDataset(train_desc, train_cvss, train_targets, tokenizer, MAX_LENGTH)
    val_dataset = NVDDataset(val_desc, val_cvss, val_targets, tokenizer, MAX_LENGTH)

    # train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, pin_memory=True, num_workers=2)

    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

    # Model, Optimizer, and Loss
    model = NVDRegressionModel(PRETRAINED_MODEL_NAME, config=config, num_numerical_features=1).to(DEVICE)
    # Resize the token embeddings to match the new vocabulary size
    if '[PAD]' in tokenizer.all_special_tokens:
      model.resize_token_embeddings(len(tokenizer))

    print("Model vocab size:", model.transformer.config.vocab_size)
    # print("Max token ID in input_ids:", .max())
    # print("Model vocab size:", model.transformer.config.vocab_size)

    optimizer = optim.AdamW(model.parameters(), lr=1e-5)
    loss_fn = nn.MSELoss()

    count = 0

    torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
    for batch in train_loader:
      count = count + 1
      input_ids = batch['input_ids']
      if input_ids.max() >= model.transformer.config.vocab_size:
        print(input_ids.max())
        raise ValueError("Token ID out of range! Check tokenizer and model vocab alignment.")

    # freeze_model_layers(model, num_layers_to_freeze=len(model.transformer.h) - 2)
    freeze_model_layers(model)

    for name, param in model.named_parameters():
      print(f"{name}: {'Trainable' if param.requires_grad else 'Frozen'}")


    # Training Loop
    for epoch in range(EPOCHS):
        train_loss = train(model, train_loader, optimizer, loss_fn, DEVICE)
        print(f"Epoch {epoch + 1}, Train Loss: {train_loss:.4f}")

    # Evaluation
    predictions, actuals = evaluate(model, val_loader, DEVICE)
    print("Sample Predictions:", predictions[:5])
    print("Sample Actuals:", actuals[:5])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Tokenizer vocab size: 32000
Model vocab size: 32000
Layer: transformer, Type: <class 'transformers.models.llama.modeling_llama.LlamaForCausalLM'>
Layer: linear, Type: <class 'torch.nn.modules.linear.Linear'>
2
Froze 0 out of 2 transformer layers.
transformer.model.embed_tokens.weight: Trainable
transformer.model.layers.0.self_attn.q_proj.weight: Trainable
transformer.model.layers.0.self_attn.k_proj.weight: Trainable
transformer.model.layers.0.self_attn.v_proj.weight: Trainable
transformer.model.layers.0.self_attn.o_proj.weight: Trainable
transformer.model.layers.0.mlp.gate_proj.weight: Trainable
transformer.model.layers.0.mlp.up_proj.weight: Trainable
transformer.model.layers.0.mlp.down_proj.weight: Trainable
transformer.model.layers.0.input_layernorm.weight: Trainable
transformer.model.layers.0.post_attention_layernorm.weight: Trainable
transformer.model.layers.1.self_attn.q_proj.weight: Trainable
transformer.model.layers.1.self_attn.k_proj.weight: Trainable
transformer.model.layers.1

Processing:   0%|          | 0/328 [00:00<?, ?it/s]

CVSS Scores: tensor([7.8000, 7.8000, 8.8000, 9.8000, 4.7000], device='cuda:0')
Targets: tensor([1.8000, 1.8000, 2.8000, 3.9000, 1.0000], device='cuda:0')


<ipython-input-2-0c03b16f9587>:114: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
LlamaModel is using LlamaSdpaAttention, but `torch.nn.functional.scaled_dot_product_attention` does not support `output_attentions=True`. Falling back to the manual attention implementation, but specifying the manual implementation will be required from Transformers version v5.0.0 onwards. This warning can be removed using the argument `attn_implementation="eager"` when loading the model.
Processing:   0%|          | 0/328 [00:01<?, ?it/s]


ValueError: Model didnt return hidden state

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, explained_variance_score
import numpy as np

def compute_regression_metrics(predictions, actuals):
    # Mean Absolute Error
    mae = mean_absolute_error(actuals, predictions)

    # Mean Squared Error
    mse = mean_squared_error(actuals, predictions)

    # Root Mean Squared Error
    rmse = np.sqrt(mse)

    # R-squared
    r2 = r2_score(actuals, predictions)

    # Explained Variance Score
    evs = explained_variance_score(actuals, predictions)

    # Print metrics
    print(f"Mean Absolute Error (MAE): {mae:.4f}")
    print(f"Mean Squared Error (MSE): {mse:.4f}")
    print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")
    print(f"R-squared (R²): {r2:.4f}")
    print(f"Explained Variance Score: {evs:.4f}")

    return {"MAE": mae, "MSE": mse, "RMSE": rmse, "R²": r2, "Explained Variance Score": evs}

# Example usage
predictions, actuals = evaluate(model, val_loader, DEVICE)
print("Sample Predictions:", predictions[:5])
print("Sample Actuals:", actuals[:5])

# Compute regression metrics
metrics = compute_regression_metrics(predictions, actuals)


Sample Predictions: [1.8329941, 2.751694, 2.1770105, 2.1772223, 1.8384278]
Sample Actuals: [2.8, 2.8, 3.9, 3.9, 2.3]
Mean Absolute Error (MAE): 0.7844
Mean Squared Error (MSE): 0.9162
Root Mean Squared Error (RMSE): 0.9572
R-squared (R²): -0.1733
Explained Variance Score: -0.0854


In [ ]:
print(max(actuals))

3.9


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ValueError: Asking to pad but the tokenizer does not have a padding token. Please select a token to use as `pad_token` `(tokenizer.pad_token = tokenizer.eos_token e.g.)` or add a new pad token via `tokenizer.add_special_tokens({'pad_token': '[PAD]'})`.

In [ ]:
def convert_to_labels(exploitability_scores):
    # Define thresholds for conversion (for example)
    labels = []
    for score in exploitability_scores:
        if score < 1.5:
            labels.append(0)  # Low exploitability
        elif score < 3.2:
            labels.append(1)  # Medium exploitability
        else:
            labels.append(2)  # High exploitability
    return np.array(labels)

# Example: convert continuous scores to labels
predictions_labels = convert_to_labels(predictions)
actuals_labels = convert_to_labels(actuals)


In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix

# Calculate accuracy
accuracy = accuracy_score(actuals_labels, predictions_labels)
print(f"Accuracy: {accuracy:.4f}")

# Calculate precision, recall, and F1 score for each class
precision, recall, f1, _ = precision_recall_fscore_support(actuals_labels, predictions_labels, average=None)
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

# Confusion Matrix
conf_matrix = confusion_matrix(actuals_labels, predictions_labels)
print("Confusion Matrix:")
print(conf_matrix)


Accuracy: 0.6915
Precision: [0.         0.69436997 0.        ]
Recall: [0.         0.99615385 0.        ]
F1 Score: [0.         0.81832543 0.        ]
Confusion Matrix:
[[   0  175    0]
 [   7 1813    0]
 [   4  623    0]]


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Dataset

# Dummy dataset class
class NVDDataset(Dataset):
    def __init__(self, descriptions, cvss_scores, targets, tokenizer, max_length):
        self.descriptions = descriptions
        self.cvss_scores = cvss_scores
        self.targets = targets
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.descriptions)

    def __getitem__(self, idx):
        encoding = self.tokenizer(
            self.descriptions[idx],
            truncation=True,
            padding="max_length",
            max_length=self.max_length,
            return_tensors="pt",
        )
        return {
            "input_ids": encoding["input_ids"].squeeze(0),
            "attention_mask": encoding["attention_mask"].squeeze(0),
            "cvss_scores": torch.tensor(self.cvss_scores[idx], dtype=torch.float32),
            "target": torch.tensor(self.targets[idx], dtype=torch.float32),
        }


# Model class for regression
class NVDRegressionModel(nn.Module):
    def __init__(self, pretrained_model_name, num_numerical_features):
        super(NVDRegressionModel, self).__init__()
        self.transformer = AutoModelForSequenceClassification.from_pretrained(pretrained_model_name, num_labels=1)
        self.linear = nn.Linear(self.transformer.config.hidden_size + num_numerical_features, 1)

    def forward(self, input_ids, attention_mask, cvss_scores):
        outputs = self.transformer.bert(input_ids=input_ids, attention_mask=attention_mask)
        cls_embedding = outputs.last_hidden_state[:, 0, :]  # First token for regression
        combined_features = torch.cat((cls_embedding, cvss_scores.unsqueeze(1)), dim=1)
        output = self.linear(combined_features)
        return output.squeeze()


# Main pipeline
if __name__ == "__main__":
    # Hyperparameters
    PRETRAINED_MODEL_NAME = "distilbert/distilbert-base-uncased"
    MAX_LENGTH = 64
    BATCH_SIZE = 32
    EPOCHS = 3
    LR = 1e-5
    DEVICE = "cuda" if torch.cuda.is_available() else "cpu"



    # Tokenizer and Dataset
    tokenizer = AutoTokenizer.from_pretrained(PRETRAINED_MODEL_NAME)
    tokenizer.pad_token = tokenizer.eos_token

    train_desc, val_desc, train_cvss, val_cvss, train_targets, val_targets = train_test_split(
        descriptions, cvss_scores, targets, test_size=0.2, random_state=42
    )

    train_dataset = NVDDataset(train_desc, train_cvss, train_targets, tokenizer, MAX_LENGTH)
    val_dataset = NVDDataset(val_desc, val_cvss, val_targets, tokenizer, MAX_LENGTH)

    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

    # Model, Optimizer, and Loss
    model = NVDRegressionModel(PRETRAINED_MODEL_NAME, num_numerical_features=1).to(DEVICE)
    optimizer = optim.AdamW(model.parameters(), lr=LR)
    loss_fn = nn.MSELoss()

    # Training Loop
    for epoch in range(EPOCHS):
        model.train()
        total_loss = 0
        for batch in train_loader:
            input_ids = batch["input_ids"].to(DEVICE)
            attention_mask = batch["attention_mask"].to(DEVICE)
            cvss_scores = batch["cvss_scores"].to(DEVICE)
            targets = batch["target"].to(DEVICE)

            optimizer.zero_grad()
            predictions = model(input_ids, attention_mask, cvss_scores)
            loss = loss_fn(predictions, targets)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        print(f"Epoch {epoch + 1}, Train Loss: {total_loss / len(train_loader):.4f}")

    # Evaluation
    model.eval()
    predictions, actuals = [], []
    with torch.no_grad():
        for batch in val_loader:
            input_ids = batch["input_ids"].to(DEVICE)
            attention_mask = batch["attention_mask"].to(DEVICE)
            cvss_scores = batch["cvss_scores"].to(DEVICE)
            targets = batch["target"].to(DEVICE)

            preds = model(input_ids, attention_mask, cvss_scores)
            predictions.extend(preds.cpu().numpy())
            actuals.extend(targets.cpu().numpy())

    print("Sample Predictions:", predictions[:5])
    print("Sample Actuals:", actuals[:5])


NameError: name 'descriptions' is not defined

In [ ]:
# Save the finetuned model and tokenizer
# Save the model's state dictionary
torch.save(model.state_dict(), 'nvd_regression_model.pth')

# Compress the saved model folder into a zip file for easy downloading
import shutil
shutil.make_archive('nvd_regression_model.pth', 'zip', './nvd_regression_model.pth')


In [ ]:
from transformers import BitnetForSequenceClassification, BitnetTokenizer, Trainer, TrainingArguments

# Load BitNet tokenizer and model (make sure it's available in the Hugging Face hub)
tokenizer = BitnetTokenizer.from_pretrained('facebook/bitnet-base')
model = BitnetForSequenceClassification.from_pretrained('facebook/bitnet-base', num_labels=3)  # Assuming 3 classes for exploitability scores


In [ ]:
# prompt: gpu not free yet. colab saying 14.7 gb in use # prompt: clear the the gpu space, its filled to 14.7 gb due to a failed process. i wanna clear it back
# import torch
# # Method 1: Using torch.cuda.empty_cache()
# torch.cuda.empty_cache()
# # Method 2: Resetting peak memory stats (optional, but good practice)
# torch.cuda.reset_peak_memory_stats()
# # Optionally, check GPU memory usage again
# print(torch.cuda.memory_summary(device=None, abbreviated=False)) these didn't help

import gc

# Method 1: Delete unneeded variables
# del model, tokenizer, train_loader, val_loader, optimizer, loss_fn # etc. - delete any large variables you're no longer using

# Method 2: Force garbage collection
gc.collect()

# Method 3: Empty PyTorch CUDA cache (if applicable)
import torch
torch.cuda.empty_cache()

# Method 4: Reset peak memory statistics (optional, for monitoring)
torch.cuda.reset_peak_memory_stats()

# Optionally, check GPU memory usage again
print(torch.cuda.memory_summary(device=None, abbreviated=False))

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Dataset

# Dummy dataset class
class NVDDataset(Dataset):
    def __init__(self, descriptions, cvss_scores, targets, tokenizer, max_length):
        self.descriptions = descriptions
        self.cvss_scores = cvss_scores
        self.targets = targets
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.descriptions)

    def __getitem__(self, idx):
        encoding = self.tokenizer(
            self.descriptions[idx],
            truncation=True,
            padding="max_length",
            max_length=self.max_length,
            return_tensors="pt",
        )
        return {
            "input_ids": encoding["input_ids"].squeeze(0),
            "attention_mask": encoding["attention_mask"].squeeze(0),
            "cvss_scores": torch.tensor(self.cvss_scores[idx], dtype=torch.float32),
            "target": torch.tensor(self.targets[idx], dtype=torch.float32),
        }


# Model class for regression
class NVDRegressionModel(nn.Module):
    def __init__(self, pretrained_model_name, num_numerical_features):
        super(NVDRegressionModel, self).__init__()
        self.transformer = AutoModelForSequenceClassification.from_pretrained(pretrained_model_name, num_labels=1)
        self.linear = nn.Linear(self.transformer.config.hidden_size + num_numerical_features, 1)

    def forward(self, input_ids, attention_mask, cvss_scores):
        outputs = self.transformer.bert(input_ids=input_ids, attention_mask=attention_mask)
        cls_embedding = outputs.last_hidden_state[:, 0, :]  # First token for regression
        combined_features = torch.cat((cls_embedding, cvss_scores.unsqueeze(1)), dim=1)
        output = self.linear(combined_features)
        return output.squeeze()